### Algorithm Forward - Backward

### Alpha and beta  for the observation (2,2,3,2,3,3)

In [ ]:
import numpy as np
import pandas as pd

In [1]:
import numpy as np
import pandas as pd

P=np.array([[0.75,0.25,0,0,0],[0.25,0.5,0.25,0,0],[0,0.25,0.5,0.25,0],[0,0,0.25,0.5,0.25],[0,0,0,0.25,0.75]])
Q=np.array([[0,0,0,0,0],[0,0,0,0,0],[0,1,1,1,0],[1,0,0,0,1]])
A=np.array([1,2,3,4,5])
Ot=np.array([2,2,3,2,3,3])

def alpha(t,j,start=2) : 
    if t==0 :
        if j==start:
            return 1
        else :
            return 0
    if t==1:
        return Q[Ot[t]][j]*P[2][j]
    sum=0
    for i in range (len(A)):
        sum+=Q[Ot[t]][j]*P[i][j]*alpha(t-1,i)
    return sum

In [2]:
pd.DataFrame(Q.T)

,0,1,2,3
0,0,0,0,1
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,0,1


In [3]:

Alpha=np.zeros((len(A),len(Ot)))
for j in range (len(Ot)):
    for i in range (len(A)):
        Alpha[i][j]=alpha(j,i)
pd.DataFrame(Alpha)

,0,1,2,3,4,5
0,0.0,0.00,0.0625,0.000000,0.003906,0.00293
1,0.0,0.25,0.0000,0.015625,0.000000,0.00000
2,1.0,0.50,0.0000,0.000000,0.000000,0.00000
3,0.0,0.25,0.0000,0.015625,0.000000,0.00000
4,0.0,0.00,0.0625,0.000000,0.003906,0.00293


In [4]:
def beta(t,j,finish,start=2) : 
    if t==len(Ot)-1 :
        return 1
    sum=0
    for i in range (len(A)):
        sum+=Q[Ot[t+1]][i]*P[i][j]*beta(t+1,i,finish)
    return sum

In [5]:
Beta=np.zeros((len(A),len(Ot)))
for j in range (len(Ot)):
    for i in range (len(A)):
        Beta[i][j]=beta(j,i,len(Ot)-1)
pd.DataFrame(Beta)

,0,1,2,3,4,5
0,0.002930,0.035156,0.046875,0.5625,0.75,1.0
1,0.005859,0.011719,0.093750,0.1875,0.25,1.0
2,0.005859,0.000000,0.093750,0.0000,0.00,1.0
3,0.005859,0.011719,0.093750,0.1875,0.25,1.0
4,0.002930,0.035156,0.046875,0.5625,0.75,1.0


### Using dynamic programming

On utilise un attribut cache dans la fonction pour eviter la declaration d'un variable global

In [6]:
def alpha_dyn(t,j,start=2) : 
    if not hasattr(alpha_dyn,'cache'):
        alpha_dyn.cache= np.full((len(A),len(Ot)),-1.)
    if t==0 :
        if j==start:
            alpha_dyn.cache[j][t]=1
            return 1
        else :
            alpha_dyn.cache[j][t]=0
            return 0
    if t==1:
        alpha_dyn.cache[j][t]= Q[Ot[t]][j]*P[2][j]
        return alpha_dyn.cache[j][t]
    sum=0
    if alpha_dyn.cache[j][t]!=-1 :
        return alpha_dyn.cache[j][t]
    else :
        for i in range (len(A)):
            sum+=Q[Ot[t]][j]*P[i][j]*alpha_dyn(t-1,i)
        alpha_dyn.cache[j][t]=sum
        return sum

In [7]:
Alpha=np.zeros((len(A),len(Ot)))
for j in range (len(Ot)):
    for i in range (len(A)):
        Alpha[i][j]=alpha_dyn(j,i)
pd.DataFrame(Alpha)
 

,0,1,2,3,4,5
0,0.0,0.00,0.0625,0.000000,0.003906,0.00293
1,0.0,0.25,0.0000,0.015625,0.000000,0.00000
2,1.0,0.50,0.0000,0.000000,0.000000,0.00000
3,0.0,0.25,0.0000,0.015625,0.000000,0.00000
4,0.0,0.00,0.0625,0.000000,0.003906,0.00293


In [8]:
def beta_dyn(t,j,finish,start=2 )  : 
    if not hasattr(beta_dyn,'cache'):
        beta_dyn.cache= np.full((len(A),len(Ot)),-1.)
    if t==finish :
        beta_dyn.cache[j][t]=1
        return 1
    sum=0
    if beta_dyn.cache[j][t]!=-1 :
        return beta_dyn.cache[j][t]
    else :
        for i in range (len(A)):
            sum+=Q[Ot[t+1]][i]*P[i][j]*beta_dyn(t+1,i,finish)
        beta_dyn.cache[j][t]=sum
        return sum
    
    return sum

In [9]:
Beta=np.zeros((len(A),len(Ot)))
for j in range (len(Ot)):
    for i in range (len(A)):
        Beta[i][j]=beta_dyn(j,i,len(Ot)-1)
pd.DataFrame(Beta)

,0,1,2,3,4,5
0,0.002930,0.035156,0.046875,0.5625,0.75,1.0
1,0.005859,0.011719,0.093750,0.1875,0.25,1.0
2,0.005859,0.000000,0.093750,0.0000,0.00,1.0
3,0.005859,0.011719,0.093750,0.1875,0.25,1.0
4,0.002930,0.035156,0.046875,0.5625,0.75,1.0


### Observation decoding (2,2,2,3,3,2,2,3,3,2,2,2,3)

#### Forward-Backward

In [10]:
%%time
Ot=[2,2,2,3,3,2,2,3,3,2,2,2,3]
## clean the cache
alpha_dyn.cache= np.full((len(A),len(Ot)),-1.)
beta_dyn.cache= np.full((len(A),len(Ot)),-1.)

Pre=np.zeros((len(A),len(Ot)))
Den=0
for i in range (len(A)):
    Den+=alpha_dyn(len(Ot)-1,i)
    
for j in range (len(Ot)):
    for i in  range (len(A)):
        Pre[i][j]=(alpha_dyn(j,i)*beta_dyn(j,i,len(Ot)-1))/Den

Wall time: 2 ms


The probabilities of all states


In [11]:
pd.DataFrame(Pre)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,0.00,0.0,0.5,0.5,0.0,0.0,0.5,0.5,0.0,0.000000,0.0,0.5
1,0.0,0.25,0.5,0.0,0.0,0.5,0.5,0.0,0.0,0.5,0.333333,0.5,0.0
2,1.0,0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0
3,0.0,0.25,0.5,0.0,0.0,0.5,0.5,0.0,0.0,0.5,0.333333,0.5,0.0
4,0.0,0.00,0.0,0.5,0.5,0.0,0.0,0.5,0.5,0.0,0.000000,0.0,0.5


alphas

In [12]:
pd.DataFrame(alpha_dyn.cache)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,0.00,0.000,0.0625,0.046875,0.000000,0.000000,0.001465,0.001099,0.000000,0.000000,0.000000,0.000026
1,0.0,0.25,0.250,0.0000,0.000000,0.011719,0.005859,0.000000,0.000000,0.000275,0.000137,0.000103,0.000000
2,1.0,0.50,0.375,0.0000,0.000000,0.000000,0.005859,0.000000,0.000000,0.000000,0.000137,0.000137,0.000000
3,0.0,0.25,0.250,0.0000,0.000000,0.011719,0.005859,0.000000,0.000000,0.000275,0.000137,0.000103,0.000000
4,0.0,0.00,0.000,0.0625,0.046875,0.000000,0.000000,0.001465,0.001099,0.000000,0.000000,0.000000,0.000026


betas

In [13]:
pd.DataFrame(beta_dyn.cache)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.000013,0.000026,0.000309,0.000412,0.000549,0.001099,0.013184,0.017578,0.023438,0.03125,0.0625,0.75,1.0
1,0.000039,0.000051,0.000103,0.000137,0.001648,0.002197,0.004395,0.005859,0.078125,0.09375,0.1250,0.25,1.0
2,0.000051,0.000051,0.000000,0.000000,0.002197,0.002197,0.000000,0.000000,0.109375,0.12500,0.1250,0.00,1.0
3,0.000039,0.000051,0.000103,0.000137,0.001648,0.002197,0.004395,0.005859,0.078125,0.09375,0.1250,0.25,1.0
4,0.000013,0.000026,0.000309,0.000412,0.000549,0.001099,0.013184,0.017578,0.023438,0.03125,0.0625,0.75,1.0


### Comparison with and without dynamic programming for  (2,2,2,3,3,2,2,3,3,2)
With the 13 elements the execution takes a lot of time so we will execute just with 11 elements

In [14]:
%%time
Ot=[2,2,2,3,3,2,2,3,3,2,2]
Pre=np.zeros((len(A),len(Ot)))
Den=0
for i in range (len(A)):
    Den+=alpha(len(Ot)-1,i)
    
for j in range (len(Ot)):
    for i in  range (len(A)):
        Pre[i][j]=(alpha(j,i)*beta(j,i,len(Ot)-1))/Den

Wall time: 5min 35s


In [15]:
pd.DataFrame(Pre)

,0,1,2,3,4,5,6,7,8,9,10
0,0.0,0.00,0.0,0.5,0.5,0.0,0.0,0.5,0.5,0.0,0.000000
1,0.0,0.25,0.5,0.0,0.0,0.5,0.5,0.0,0.0,0.5,0.333333
2,1.0,0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333
3,0.0,0.25,0.5,0.0,0.0,0.5,0.5,0.0,0.0,0.5,0.333333
4,0.0,0.00,0.0,0.5,0.5,0.0,0.0,0.5,0.5,0.0,0.000000


##### So for thirteen elements:
1. with dynamic programming the execution time is 900 microseconds for 13 elements. 
1. Without dynamic programming the execution time with 11 elements is 5min 35s (for 13 elements the time is probably huge)

#### By using the predefined support for "caching decorator" of python 3

In [22]:
from functools import lru_cache

@lru_cache(maxsize=None)
def alpha(t,j,start=2) : 
    if t==0 :
        if j==start:
            return 1
        else :
            return 0
    if t==1:
        return Q[Ot[t]][j]*P[2][j]
    sum=0
    for i in range (len(A)):
        sum+=Q[Ot[t]][j]*P[i][j]*alpha(t-1,i)
    return sum

@lru_cache(maxsize=None)
def beta(t,j,finish,start=2) : 
    if t==len(Ot)-1 :
        return 1
    sum=0
    for i in range (len(A)):
        sum+=Q[Ot[t+1]][i]*P[i][j]*beta(t+1,i,finish)
    return sum


In [23]:
%%time
Ot=[2,2,2,3,3,2,2,3,3,2,2,2,3]
Pre=np.zeros((len(A),len(Ot)))
Den=0
for i in range (len(A)):
    Den+=alpha(len(Ot)-1,i)
    
for j in range (len(Ot)):
    for i in  range (len(A)):
        Pre[i][j]=(alpha(j,i)*beta(j,i,len(Ot)-1))/Den

Wall time: 2.63 ms


#### The time is very small